In [3]:
d = {
    'batch_size': 777,
    'n_devices': 4,
    'gradient_accumulation_steps': 8,
}

current_batch_size_per_gpu = d["batch_size"] // (
    d["gradient_accumulation_steps"] * d["n_devices"]
)  # This value assures that the num_batch_chunks is 1

num_batch_chunks = max(
    d["gradient_accumulation_steps"]
    // (d["batch_size"] // (current_batch_size_per_gpu * d["n_devices"])),
    1,
)

print(f"current_batch_size_per_gpu: {current_batch_size_per_gpu}")
print(f"num_batch_chunks: {num_batch_chunks}")

current_batch_size_per_gpu: 24
num_batch_chunks: 1


In [7]:
import torch

t = torch.tensor([[1, 1], [2, 2], [3, 3], [4, 4]])

chunked = torch.chunk(t, 4, dim=0)
for chunk in chunked:
    chunk[0, 1] += 7
    print(chunk)

print(t)

tensor([[1, 8]])
tensor([[2, 9]])
tensor([[ 3, 10]])
tensor([[ 4, 11]])
tensor([[ 1,  8],
        [ 2,  9],
        [ 3, 10],
        [ 4, 11]])


queisoqu9Hag